In [1]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.device('cuda')

device(type='cuda')

: 

In [ ]:
import torch
import torch.nn as nn

# class LayerNorm(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         pass

# class FlashAttention(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, q, k, v, mask=None):
#         pass

# class FeedForward(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         pass

# class DecoderBlock(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x, mask=None):
#         pass

# class RotaryPositionalEncoding(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         pass

# class Decoder(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x, mask=None):
#         pass

class LLM(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, mask=None):
        pass

def create_causal_mask():
    pass

def initialize_weights():
    pass

def load_model():
    pass

def save_model():
    pass

def generate_text():
    pass

def train_step():
    pass

def evaluate():
    pass

def main():
    pass

if __name__ == "__main__":
    main()

In [ ]:
from open_lm.hf import *
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("TRI-ML/DCLM-1B")
model = AutoModelForCausalLM.from_pretrained("TRI-ML/DCLM-1B")

inputs = tokenizer(["Machine learning is"], return_tensors="pt")

In [1]:
from threading import local


def flash_attention(q,k,v,mask=None,dropout=None,block_size=1024):
    #extractttt info from query tensor
    batch_size,num_heads,seq_len,d_k=q.size()
    #scaling factor for the dot product attention
    scale=1/math.sqrt(d_k)
    #intializee tensors to store final output and atteention weights
    output=torch.zeros_like(q)
    attention_weights=torch.zeros(batch_size,seq_len,seq_len,device=q.device)

    #itterate over the sequence in block s
    for block_start in range(0,seq_len,block_size):
        #caclulate the end of the block
        block_end=min(block_start+block_size,seq_len)
        
        #extracl block for query key and values 
        local_q=q[::,:,block_start:block_end]
        local_k=k[:,:,:block_end]
        local_v=v[:,:,:block_end]

        #local attention scaled dod product fo it is calcualted
        local_attention=torch.matmul(local_q,local_k.transpose(-1,-2))*scale

        #apply mask if provided
        if mask is not None:
            local_attn=local_attn+mask[:,:,block_start:block_end,:block_end]

        #apply softmax
        local_attn=F.softmax(local_attn,dim=-1)

        #apply dropout if provided 
        if dropout is not None:
            local_attn=dropout(local_attn)

        #compute output of current block
        local_output=torch.matmul(local_attn,local_v)

        #store output of current block
        output[:,:,block_start:block_end]=local_output
        attention_weights[:,:,block_start:block_end]=local_attn

    return output,attention_weights    

def flash_scaled_dot_product(q,k,v,mask):
    d_k=q.size()[:-1]
    values,attention=flash_attention(q,k,v,mask)
    return values,attention


: 

In [ ]:
class rotaryembedding(nn.Module):
    def __init__(self,dim):
        super().__init__()
        self.dim=dim
    def forward(self,seq_len):
        #asmuch as i understand thsi is cuz we add the psostionsal emvbeddings for evene and odd seeperatelty so calcuating it for half the dimentionality
        half_dim=self.dim//2
        #base for exponential decay 
        emb=math.log(10000)/(half_dim-1)
        #basically wavelenghts that increase exponentially
        emb=torch.exp(torch.arange(half_dim,device=self.device)*-emb)
        #outer prodcut creates sine waves
        emb=torch.outer(torch.arange(seq_len,device=self.device),emb)    
        #this creates pairs of orthogonal vector for each psoition
        emb=torch.stack([torch.cos(emb),torch.sin(emb)],dim=-1)
        #unsqueeze like devides vector to size one dim 
        return emb.unsqueeze(1)

class RotaryPosEnc(nn.Module):
    def __init__(self,d_model,max_sequence_length):
        super().__init__()
        self.max_sequence_length=max_sequence_length
        self.d_model=d_model
        self.rotary_emb=rotaryembedding(self.d_model//2)
    def forward(self,x):
        pos_emb=self.rotary_emb(self.max_sequence_length)
        cos_pos,sin_pos=pos_emb.unbind(dim=-1)
        x_even,x_odd=x.chunk(2,dim=-1)
        x_even_new=x_even*cos_pos-x_odd-sin_pos
        x_odd_new=x_odd*cos_pos-x_even*sin_pos

        x_new=torch.stack([x_even_new,x_odd_new],dim=-1)
        return x_new.flatten(2)

In [ ]:
class Decoderlayer(nn.Module):
    def __init__(self,d_model,num_heads,d_ff,dropout):
        super().__init__()
        self.self_attn=FlashAttention(d_model,num_heads)
        self.norm1=LayerNorm(d_model)
        self.feed_forward=FeedForward(d_model,d_ff)
        self.norm2=LayerNorm(d_model)
        self.dropout=nn.Dropout(dropout)
    def forward(self,x,mask=None):
        attn_output=self.self_attn(x,x,x,mask)
        x=self.norm1(x+self.dropout(attn_output))
        ff_output=self.feed_forward(x)
        x=self.norm2(x+self.dropout(ff_output))
        return x


In [ ]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,d_model,num_heads,num_layers,d_ff,max_sequence_length,dropout=0.1):
        super().__init__()
        self.token_embeddings=nn.Embedding(vocab_size,d_model) 
        self.rotary_pe=RotaryPositionalEncoding(d_model,max_sequence_length)
        self.layers=nn.ModuleList([
            Decoderlayer(d_model,num_heads,d_ff,dropout)
            for _ in range(num_layers)
            ])

        self.layer_norm=LayerNorm(d_model)
        self.ouput_projections=nn.Linear(d_model,vocab_size)
        self.dropout=nn.Dropout(dropout)
    def forward(self,x,mask=None):
        x=self.token_embeddings(x)
        x=self.rotary_pe(x)
        x=self.dropout(x)
        for layer in self.layers:
            x=layer(x,mask)
        x=self.layer_norm(x)
        logits=self.output_projections(x)
        return logits        

In [ ]:
def LayerNorm(nn.Module):
    def __init__(self,d_model,epsilon=1e-5):
        super().__init__()
        self.gamma=nn.Parameter(torch.ones(d_model))
        # Parameters are Tensor subclasses, that have a very special property when used with Module s - when they’re assigned as Module attributes they are automatically added to the list of its parameters, and will appear e.g. in parameters() iterator. Assigning a Tensor doesn’t have such effect. This is because one might want to cache some temporary state, like last hidden state of the RNN, in the model. If there was no such class as Parameter, these temporaries would get registered too.
        self.beta=nn.parameter(torch.zeros(d_model))
        self.epsilon=epsilon
    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)
        var=x.var(dim=-1,keepdim=True,unbiased=False)
        x_norm=(x-mean)/torch.sqrt(var+self.epsilon)
        #these learnable parameters adjust themselves to mayeb undo the normalization if needed 
        out=self.gamma*x_norm+self.beta
        return out

In [ ]:


class FeedForward(nn.Module):
    def __init__(self,d_model,d_ff.dropout=0.1,activation='gelu'):
        super().__init__()
        self.d_model=d_model
        self.dropout=dropout
        self.linear1=nn.Linear(d_model,d_ff)
        self.linear2=nn.Linear(d_ff,d_model)
        self.dropout_layer=nn.Dropout(dropout)
        self.activation=F.gelu

    def forward(self,x):
        x=self.linear1(x)
        x=self.activation(x)
        x=self.dropout_layer(x)
        x=self.linear2(x)
        return x





In [ ]:
class LL(nn.Module):
    def __init__(self,vocab_size,d_model,num_heads,num_layers,d_ff,max_seq_leangth,dropout=0.1):
        super().__init__()
        self.token_embedding=nn.Embedding(vocab_size,d_model)
        self.rotary_pe=RotaryPosEnc(d_model,max_seq_leangth)
        self.decoder=Decoder(d_model=d_model,vocab_size=vocab_size,max_sequence_length=max_seq_leangth,num_heads=num_heads,num_layers=num_layers,d_ff=d_ff,dropout=dropout)
        self.final_layer_norm=LayerNorm(d_model)
        self.ouput_projection=nn.Linear(d_model,vocab_size)

    def forward(self,input_ids,attention_mask=None):
        x=self.token_embeding(input_ids)
        x=self.rotary_pe(x)
        x=self.decoder(x,mask=attention_mask)
        x=self.final_layer_norm(x)
        logits=self.ouput_projection(x)
        return logits   
    def generate(self,input_ids,max_length,temperature=1.0):
        self.eval()
        with torch.no_grad():
            for _ in range(max_length-input_ids.size(1)):
                outputs=self(input_ids)
                next_token_logits=outputs[:,-1,:]/temperature
                next_token_probs=torch.softmax(next_token_logits,dim=-1)
                next_token=torch.multinomial(next_token_probs,num_samples=1)
                input_ids=torch.cat([input_ids,next_token],dim=-1)
                if next_token.item()==self.eos_token_id:
                    break
        return input_ids        

In [ ]:
def load_model(model,path):
    model.load_state_dict(torch.load(path))
    return model
def save_model(model,path):
    torch.save(model.state_dict(),path)
def initialize_weights(model):
    for p in model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
def train_step(model,optimizer,batch,loss_fn):
    model.train()
    optimizer.zero_grad()
    input_ids=batch['input_ids']
    labels=batch['labels']
    attention_mask=batch['attention_mask']
    outputs=model(input_ids,attention_mask=attention_mask)
    loss=loss_fn(outputs.view(-1,outputs.size(-1)),labels.view(-1))
    loss.backward()
    optimizer.step()
    return loss.item()
def evaluate(model,dataloader,loss_fn):
    model.eval()
    total_loss=0
    total_count=0

    with torch.no_grad():
        for batch in dataloader:
            input_ids=batch['input_ids']
            labels=batch['labels']
            attention_mask=['attention_mask']
            outputs=model(input_ids,attention_mask=attention_mask)
            loss=loss_fn(outputs.view(-1,outputs.size(-1),labels.view(-1)))
            total_loss+=loss.item()*input_ids.size(0)
            total_count+=input_ids.size(0)
    return total_loss/total_count        